## Structured Output 

#### Models can be requested to provide their response in a format matching a given schema . this is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

## Pydantic 
- Helps define structured output schemas
- Types
    - Pydantic (**field validation**, type checking , json schema generation, nested structures)
    - Typedict
    - DataClasses

In [ ]:
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:llama-3.1-8b-instant")
model


/Users/adityasingh/Projects/GEN AI INT/langChain/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x111fdd2b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111fddd30>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year the movie was released")
    director: str = Field(description="The director of the movie")
    rating: float = Field(description="The movie rating out of 10")
    

In [3]:
model_with_Structure = model.with_structured_output(Movie)
model_with_Structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x111fdd2b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111fddd30>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movie rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'director',

In [ ]:
model.invoke("Provide the details about the movie Inception") # Normal response

AIMessage(content='Inception is a 2010 science fiction action film written and directed by Christopher Nolan. The film is a complex and layered narrative that explores the concept of shared dreaming, where a team of thieves specialize in entering people\'s dreams and stealing their secrets.\n\n**Plot**\n\nThe film follows Cobb (Leonardo DiCaprio), a skilled thief who specializes in entering people\'s dreams and stealing their secrets. Cobb is haunted by the memory of his wife, Mal (Marion Cotillard), who died in his arms during a dream state. Cobb\'s team, known as "extractors," is hired by a wealthy businessman named Saito (Ken Watanabe) to perform a task known as "inception" – planting an idea in someone\'s mind instead of stealing one.\n\nSaito wants Cobb to convince Robert Fischer (Cillian Murphy), the son of a dying business magnate, to dissolve his father\'s company. In order to do this, Cobb must assemble a team of experts, including Arthur (Joseph Gordon-Levitt), Ariadne (Ellen

In [ ]:
model_with_Structure.invoke("Provide the details about the movie Inception") # Structured response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

### Message output alongside parsed structure

In [6]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year the movie was released")
    director: str = Field(description="The director of the movie")
    rating: float = Field(description="The movie rating out of 10")



In [7]:
model_with_Structure = model.with_structured_output(Movie, include_raw=True) #include raw response along with structured response
model_with_Structure.invoke("Provide the details about the movie Inception")

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'y1jxfa6bh', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.5,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 280, 'total_tokens': 313, 'completion_time': 0.041682655, 'completion_tokens_details': None, 'prompt_time': 0.019011392, 'prompt_tokens_details': None, 'queue_time': 0.045877877, 'total_time': 0.060694047}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6cd7-d1c5-77d2-bc1e-eb1e6d41ca0e-0', tool_calls=[{'name': 'Movie', 'args': {'director': 'Christopher Nolan', 'rating': 8.5, 'title': 'Inception', 'year': 2010}, 'id': 'y1jxfa6bh', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 280, 'output_tokens': 33, 't

### Nested Structured Output

In [8]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name:str=Field(description="The name of the actor")
    role:str=Field(description="The role of the actor")

class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None, description="Budget in millions of USD")
    

In [ ]:
model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Tell me about The Godfather")
print(response)

{'director': 'Francis Ford Coppola', 'ratings': 9.2, 'title': 'The Godfather', 'year': 1972}


### TypedDict
##### TypedDict provides a simpler alternative using python's built-in typing , ideal when you don't need runtime validation

In [12]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details"""
    title: Annotated[str, ...,"The title of the movie"]
    year: Annotated[int, ...,"The year the movie was released"]
    director: Annotated[str, ...,"The director of the movie"]
    ratings: Annotated[float | None, ...,"The ratings of the movie"]

In [ ]:
model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Tell me about The Godfather")
print(response) # return dictionary


{'director': 'Francis Ford Coppola', 'ratings': 9.2, 'title': 'The Godfather', 'year': 1972}


In [15]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 8192,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': False,
 'tool_calling': True}

## DataClasses

#### A data class is a class typically containing mainly data,there aren't really any restrictions . You can create it using @dataclass decorator

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")



In [17]:
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """Contact information for a person"""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model=ChatGroq(model="llama-3.1-8b-instant"),
    response_format=ContactInfo # Auto selects ProviderStrategy
)

results = agent.invoke({"messages": [{"role": "user", "content": "Extract: name: John Doe, email: john@example.com, phone: 123-456-7890"}]})
results



{'messages': [HumanMessage(content='Extract: name: John Doe, email: john@example.com, phone: 123-456-7890', additional_kwargs={}, response_metadata={}, id='b4cc5426-d887-4dd0-8dab-f60dea7fed41'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'w0ak78phy', 'function': {'arguments': '{"email":"john@example.com","name":"John Doe","phone":"123-456-7890"}', 'name': 'ContactInfo'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 290, 'total_tokens': 322, 'completion_time': 0.032369818, 'completion_tokens_details': None, 'prompt_time': 0.018049015, 'prompt_tokens_details': None, 'queue_time': 0.045699514, 'total_time': 0.050418833}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6d3a-c193-71c3-b3c8-287b5570ea18-0', tool_calls=[{'name': 'ContactInfo', 'args': {'email': 'john@e

In [18]:
results['structured_response']

ContactInfo(name='John Doe', email='john@example.com', phone='123-456-7890')

In [20]:
### DATACLASS

from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    """Contact information for a person"""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person

agent = create_agent(
    model=ChatGroq(model="llama-3.1-8b-instant"),
    response_format=ContactInfo
)

results = agent.invoke({"messages": [{"role": "user", "content": "Extract: name: John Doe, email: john@example.com, phone: 123-456-7890"}]})
results["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='123-456-7890')